In [1]:
import os
import shutil
import numpy as np
import random

source_path = 'dataset'
destination_path = 'dataset_50_per'

def create_structure():
    if not os.path.exists(destination_path):
        os.makedirs(destination_path)
    if not os.path.exists(os.path.join(destination_path, 'images')):
        os.makedirs(os.path.join(destination_path, 'images'))
    if not os.path.exists(os.path.join(destination_path, 'masks')):
        shutil.copytree(os.path.join(source_path, 'masks'), os.path.join(destination_path, 'masks'))

def process_npy_tif(satellite):
    satellite_path = os.path.join(source_path, 'images', satellite)
    dest_sat_path = os.path.join(destination_path, 'images', satellite)
    
    for data_type in ['npy', 'tif']:
        data_path = os.path.join(satellite_path, data_type)
        dest_data_path = os.path.join(dest_sat_path, data_type)
        
        if not os.path.exists(dest_data_path):
            os.makedirs(dest_data_path)
        
        for folder in os.listdir(data_path):
            folder_path = os.path.join(data_path, folder)
            dest_folder_path = os.path.join(dest_data_path, folder)
            
            if data_type == 'npy':
                npz_files = os.listdir(folder_path)
                selected_files = random.sample(npz_files, len(npz_files) // 2)
                os.makedirs(dest_folder_path)
                for file in selected_files:
                    shutil.copy2(os.path.join(folder_path, file), dest_folder_path)
            else:
                # 'tif' processing based on remaining 'npy' selection
                corresponding_npy_path = os.path.join(dest_sat_path, 'npy', folder)
                if os.path.exists(corresponding_npy_path):
                    selected_folders = [f[:-4] for f in os.listdir(corresponding_npy_path)]  # assuming the names match minus .npz
                    os.makedirs(dest_folder_path)
                    for subfolder in selected_folders:
                        subfolder_path = os.path.join(folder_path, subfolder)
                        dest_subfolder_path = os.path.join(dest_folder_path, subfolder)
                        if os.path.exists(subfolder_path):
                            shutil.copytree(subfolder_path, dest_subfolder_path)

create_structure()
for satellite in ['L8', 'S2']:
    process_npy_tif(satellite)

In [1]:
source_path = 'dataset'
destination_path = 'dataset_20_per'

def create_structure():
    if not os.path.exists(destination_path):
        os.makedirs(destination_path)
    if not os.path.exists(os.path.join(destination_path, 'images')):
        os.makedirs(os.path.join(destination_path, 'images'))
    if not os.path.exists(os.path.join(destination_path, 'masks')):
        shutil.copytree(os.path.join(source_path, 'masks'), os.path.join(destination_path, 'masks'))

def process_npy_tif(satellite):
    satellite_path = os.path.join(source_path, 'images', satellite)
    dest_sat_path = os.path.join(destination_path, 'images', satellite)
    
    for data_type in ['npy', 'tif']:
        data_path = os.path.join(satellite_path, data_type)
        dest_data_path = os.path.join(dest_sat_path, data_type)
        
        if not os.path.exists(dest_data_path):
            os.makedirs(dest_data_path)
        
        for folder in os.listdir(data_path):
            folder_path = os.path.join(data_path, folder)
            dest_folder_path = os.path.join(dest_data_path, folder)
            
            if data_type == 'npy':
                npz_files = os.listdir(folder_path)
                selected_files = random.sample(npz_files, len(npz_files) // 5)
                os.makedirs(dest_folder_path)
                for file in selected_files:
                    shutil.copy2(os.path.join(folder_path, file), dest_folder_path)
            else:
                # 'tif' processing based on remaining 'npy' selection
                corresponding_npy_path = os.path.join(dest_sat_path, 'npy', folder)
                if os.path.exists(corresponding_npy_path):
                    selected_folders = [f[:-4] for f in os.listdir(corresponding_npy_path)]  # assuming the names match minus .npz
                    os.makedirs(dest_folder_path)
                    for subfolder in selected_folders:
                        subfolder_path = os.path.join(folder_path, subfolder)
                        dest_subfolder_path = os.path.join(dest_folder_path, subfolder)
                        if os.path.exists(subfolder_path):
                            shutil.copytree(subfolder_path, dest_subfolder_path)

create_structure()
for satellite in ['L8', 'S2']:
    process_npy_tif(satellite)

In [1]:
import os

# Define datasets and tasks
datasets = ["dataset", "dataset_20_per", "dataset_50_per"]
tasks = ["crop_type"]  # Add more tasks here if needed
base_run_path = "runs/wacv_2024_seed0"

# Function to generate the shell commands
def generate_shell_script(output_file="run_eval.sh"):
    # Ensure the output_results_verbose folder exists
    os.makedirs("output_results_verbose", exist_ok=True)

    with open(output_file, 'w') as f:
        f.write("#!/bin/bash\n\n")

        # Loop over tasks and datasets to generate commands
        for task in tasks:
            for dataset in datasets:
                for dataset_2 in datasets:
                    base_path = f"{base_run_path}/{task}"

                    # CONCAT case (fusion = 0)
                    run_name_concat = f"evaluate_test_seed0_batch_{task}_CONCAT_{dataset}_seed0_1_{dataset_2}"
                    best_path_concat = f"{base_path}/CONCAT_{dataset}_seed0_1"
                    output_file_concat = f"output_results_verbose/{run_name_concat}_output.txt"
                    f.write(f'python evaluate.py --data_dir ./{dataset_2} --satellite [S1,S2,L8] --task {task} --run_name "{run_name_concat}" --model utae --fusion 0 --best_path "{best_path_concat}" > {output_file_concat} 2>&1\n')

                    # CROSS_ALL_NORM_DROP case (fusion = 12)
                    # run_name_norm_drop = f"evaluate_test_seed0_batch_{task}_CROSS_ALL_NORM_DROP_{dataset}_seed0_1_{dataset_2}"
                    # best_path_norm_drop = f"{base_path}/CROSS_ALL_NORM_DROP_{dataset}_seed0_1"
                    # output_file_norm_drop = f"output_results_verbose/{run_name_norm_drop}_output.txt"
                    # f.write(f'python evaluate.py --data_dir ./{dataset_2} --satellite [S1,S2,L8] --task {task} --run_name "{run_name_norm_drop}" --model utae --fusion 12 --best_path "{best_path_norm_drop}" > {output_file_norm_drop} 2>&1\n')

                    # CROSS_ALL_NORM_BATCH_DROP case (fusion = 14)
                    run_name_batch_drop = f"evaluate_test_seed0_batch_{task}_CROSS_ALL_NORM_BATCH_DROP_{dataset}_seed0_1_{dataset_2}"
                    best_path_batch_drop = f"{base_path}/CROSS_ALL_NORM_BATCH_DROP_{dataset}_seed0_1"
                    output_file_batch_drop = f"output_results_verbose/{run_name_batch_drop}_output.txt"
                    f.write(f'python evaluate.py --data_dir ./{dataset_2} --satellite [S1,S2,L8] --task {task} --run_name "{run_name_batch_drop}" --model utae --fusion 14 --best_path "{best_path_batch_drop}" > {output_file_batch_drop} 2>&1\n')

        print(f"Shell script '{output_file}' has been generated successfully.")

# Call the function to generate the script
if __name__ == "__main__":
    generate_shell_script()


Shell script 'run_eval.sh' has been generated successfully.


In [2]:
import os

# Define datasets and tasks
# datasets = ["dataset", "dataset_20_per", "dataset_50_per"]
tasks = ["harvesting_date"]  # Add more tasks here if needed
base_run_path = "runs/wacv_2024_seed"

# Function to generate the shell commands
def generate_shell_script(output_file="run_eval_harvest.sh"):
    # Ensure the output_results_verbose folder exists
    os.makedirs("output_results_verbose", exist_ok=True)

    with open(output_file, 'w') as f:
        f.write("#!/bin/bash\n\n")

        # Loop over tasks and datasets to generate commands
        for seed in range(0,1):
            for task in tasks:
                for dataset_2 in ['dataset']: # test
                    for dataset in ['dataset_50_per']:
                        base_path = f"{base_run_path+str(seed)}/{task}"

                        # CONCAT case (fusion = 0)
                        run_name_concat = f"evaluate_test_seed0_batch_{task}_CONCAT_{dataset}_seed{seed}_1_{dataset_2}_unet3d"
                        best_path_concat = f"{base_path}/CONCAT_{dataset}_seed{seed}_1_unet3d"
                        output_file_concat = f"output_results_verbose/{run_name_concat}_output.txt"
                        f.write(f'python evaluate.py --data_dir ./{dataset_2} --satellite [S1,S2,L8] --task {task} --run_name "{run_name_concat}" --model utae --fusion 0 --best_path "{best_path_concat}" > {output_file_concat} 2>&1\n')

                        # # CROSS_ALL_NORM_DROP case (fusion = 12)
                        # run_name_norm_drop = f"evaluate_test_seed0_batch_{task}_CROSS_ALL_NORM_DROP_{dataset}_seed{seed}_1_{dataset_2}_unet3d"
                        # best_path_norm_drop = f"{base_path}/CROSS_ALL_NORM_DROP_{dataset}_seed{seed}_1_unet3d"
                        # output_file_norm_drop = f"output_results_verbose/{run_name_norm_drop}_output.txt"
                        # f.write(f'python evaluate.py --data_dir ./{dataset_2} --satellite [S1,S2,L8] --task {task} --run_name "{run_name_norm_drop}" --model utae --fusion 12 --best_path "{best_path_norm_drop}" > {output_file_norm_drop} 2>&1\n')

                        # CROSS_ALL_NORM_BATCH_DROP case (fusion = 14)
                        run_name_batch_drop = f"evaluate_test_seed0_batch_{task}_CROSS_ALL_NORM_BATCH_DROP_{dataset}_seed{seed}_1_{dataset_2}_unet3d"
                        best_path_batch_drop = f"{base_path}/CROSS_ALL_NORM_BATCH_DROP_{dataset}_seed{seed}_1"
                        output_file_batch_drop = f"output_results_verbose/{run_name_batch_drop}_output.txt"
                        f.write(f'python evaluate.py --data_dir ./{dataset_2} --satellite [S1,S2,L8] --task {task} --run_name "{run_name_batch_drop}" --model unet3d --fusion 14 --best_path "{best_path_batch_drop}" > {output_file_batch_drop} 2>&1\n')

        print(f"Shell script '{output_file}' has been generated successfully.")

# Call the function to generate the script
if __name__ == "__main__":
    generate_shell_script()


Shell script 'run_eval_harvest.sh' has been generated successfully.
